In [1]:
#####################################################
# Author: Prof. Marc Jofre
# Dept. Network Engineering
# Universitat Politècnica de Catalunya - Technical University of Catalonia
#
# Modified: 2024
# Created: 2024
#
# Script Quantum LAN application Client layer
#####################################################
import sys, os

pathScriptBelowAgentScript='./CppScripts/'
sys.path.append(pathScriptBelowAgentScript)
pathScriptBelowAgentScript='./JupyterPythonScripts/'
sys.path.append(pathScriptBelowAgentScript)

import QpresentationLayerAgent
#####################################################
# Parameters
## Network
IPhostClientOpNet = "10.0.0.2" # IP address of the client host (this host) in the operation network
IPnodeClientConNet = "192.168.8.2" # IP address of the client node (connected to the client host) in the control/configuration network
IPhostServerOpNet = "10.0.0.3" # IP address of the server host in the operation network
IPnodeServerConNet = "192.168.9.2" # IP address of the server node in the control/configuration network

ParamsClientListChar = [IPnodeClientConNet,IPhostServerOpNet,IPhostClientOpNet,"client"]

In [2]:
##############################################################
# Functions
def ListCharArrayParser(ListCharArrayAux):
    # Actually concatenating a python list of strings to a single string
    ParsedCharArrayAux=",".join(ListCharArrayAux)
    return ParsedCharArrayAux

In [3]:
# The parameters passed and received from agents are structured
ParamsAscendingCharArray = "none"
ParamsDescendingCharArray = ListCharArrayParser(ParamsClientListChar)

In [4]:
### RUN UP TO HERE (INCLUDED) TO INITIALIZE CONNECTIONS AND THREAD QtransportLayerAgentH thread #####
# At this point, in this order, the node in the server has been initialized, the node in the client, the server host (up to this equivalent point) and then the client host (up to here)
# Create instance of Agent below
QPLAagent = QpresentationLayerAgent.QPLA(ParamsDescendingCharArray,ParamsAscendingCharArray)
### Initiate thread process in QtransportLayerAgentH
QPLAagent.InitAgentProcess() # Initialize the thread in QtransportLayerAgentH

In [6]:
#####################
# Some tests to send information to the different elements and see them output at the respective console
# The required fields of a message are (separated by commas): IPdestination,IPorigin,Type,Command,Payload
messagePayloadAux="Host client sends information to attached node"
messageCommandAux="print"
messageTypeAux="Info"
messageIPorg=IPhostClientOpNet
messageIPdest=IPnodeClientConNet
messageAuxChar = ListCharArrayParser([messageIPdest,messageIPorg,messageTypeAux,messageCommandAux,messagePayloadAux])
QPLAagent.SendMessageAgent(messageAuxChar)

messagePayloadAux="Host client sends information to server host"
messageCommandAux="print"
messageTypeAux="Info"
messageIPorg=IPhostClientOpNet
messageIPdest=IPhostServerOpNet
messageAuxChar = ListCharArrayParser([messageIPdest,messageIPorg,messageTypeAux,messageCommandAux,messagePayloadAux])
QPLAagent.SendMessageAgent(messageAuxChar)

messagePayloadAux="Host client sends information to server node"
messageCommandAux="print"
messageTypeAux="Control"
messageIPorg=IPhostClientOpNet
messageIPdest=IPhostServerOpNet # Notice that it is send to the host (which it will forward it to the node)
messageAuxChar = ListCharArrayParser([messageIPdest,messageIPorg,messageTypeAux,messageCommandAux,messagePayloadAux])
QPLAagent.SendMessageAgent(messageAuxChar)
